In [19]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import time

In [20]:
df = pd.read_csv('01_bus_routes.csv')

In [21]:
all_bus_details = []

#XPATH
xpath_bus_details = '//div[@class="clearfix row-one"]'

#create a chrome driver instance
driver = webdriver.Chrome()

#defines to wait until a certain operation
wait = WebDriverWait(driver,10) #10 seconds dynamic wait
wait_ = WebDriverWait(driver,5) 

In [22]:
#Helper function to select view_buses
def view_buses():
    xpath_view_buses = '//div[@class="button" and text()="View Buses"]'
    
    try:
        object_viewbuses = wait.until(EC.presence_of_all_elements_located((By.XPATH,xpath_view_buses)))
    except (NoSuchElementException,TimeoutException):
        return
    
    if object_viewbuses:
        for element in reversed(object_viewbuses): #reversed() to click button from bottom so, following buttons will be in sight
                    try:
                        time.sleep(2)
                        element.click()
                        time.sleep(2) # 2 sec wait to load dynamic content
                    except Exception:
                        continue

In [23]:
#helper function to scroll bottom of page dynamically
def scroll_to_bottom():
    while True:
        # get initial page height
        current_page_height = driver.execute_script("return document.body.scrollHeight")

        #scroll to bottom of page
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        
        # Wait for the page to load new content...if any
        time.sleep(1)

        #get new height after scrolling page
        currentpage_new_height = driver.execute_script("return document.body.scrollHeight") 

        if currentpage_new_height == current_page_height:
            break
        
        current_page_height = currentpage_new_height


In [24]:
# for each iteration it scrapes current page data
def scrape_current_page(state, route, link, elements_data):
    bus_details = []
    for element in elements_data:
        try:
            bus_name = element.find_element(By.XPATH, './/div[@class="travels lh-24 f-bold d-color"]').text
        except NoSuchElementException:
            bus_name = "NA"

        try:
            bus_type = element.find_element(By.XPATH, './/div[@class="bus-type f-12 m-top-16 l-color evBus"]').text
        except NoSuchElementException:
            bus_type = "NA"

        try:
            departing_time = element.find_element(By.XPATH, './/div[@class="dp-time f-19 d-color f-bold"]').text
        except NoSuchElementException:
            departing_time = "NA"

        try:
            duration = element.find_element(By.XPATH, './/div[@class="dur l-color lh-24"]').text
        except NoSuchElementException:
            duration = "NA"

        try:
            reaching_time = element.find_element(By.XPATH, './/div[@class="bp-time f-19 d-color disp-Inline"]').text
        except NoSuchElementException:
            reaching_time = "NA"

        try:
            price = element.find_element(By.XPATH, './/span[contains(@class, "f-19 f-bold") or contains(@class, "f-bold f-19")]').text
        except NoSuchElementException:
            price = "NA"

        try:
            star_rating = element.find_element(By.XPATH, './/div[@class="rating-sec lh-24"]//span').text
        except NoSuchElementException:
            try:
                element.find_element(By.XPATH, './/span[contains(@class, "blue rating_badge")]')
                star_rating = '0'
            except NoSuchElementException:
                star_rating = '0'

        try:
            seat_available = element.find_element(By.XPATH, './/div[contains(@class, "seat-left") and (contains(@class, "m-top-16") or contains(@class, "m-top-30"))]').text
        except NoSuchElementException:
            seat_available = "NA"

        bus_details.append([state, route, link, bus_name, bus_type, departing_time, duration, reaching_time, price, star_rating, seat_available])
    
    return bus_details


In [25]:
def check_oops_message():
    # check_xpath = "//div[@class='oops-wrapper']//h3[text()='Oops! No buses found.']"
    check_xpath = (
        "//div[(contains(@class, 'oops-wrapper') and //h3[text()='Oops! No buses found.']) or "
        "(@class='oops-wrapper new_oops_wrapper' and text()='Oops! No buses found.')]"
    )
    try:
        wait_.until(EC.presence_of_element_located((By.XPATH, check_xpath)))
        return True
    except (NoSuchElementException,TimeoutException):
        return False

In [26]:

def main_function():
    for _,row in df.iterrows():
        state = row['state_names']
        route = row['routes']
        link = row['links']

        try:
            driver.get(link)
            
            if check_oops_message():
                continue

            driver.maximize_window()
            view_buses()  #click on button view_buses
            scroll_to_bottom()  #scroll pages dynamically to bottom

            elements_data = wait_.until(EC.presence_of_all_elements_located((By.XPATH,xpath_bus_details)))

            if elements_data:
                current_page_data = scrape_current_page(state, route, link, elements_data)
                all_bus_details.extend(current_page_data)
            else:
                print(f"No buses available for state: {state}, route: {route}, link: {link}")    
        except Exception as e:
            print(f"Error on line: state: {state}, route: {route}, link: {link}")
            print(f"Error_details: {e}")         

    driver.quit()

In [ ]:
main_function()
final_data = pd.DataFrame(all_bus_details,columns=['state','route','link','busname','bustype','departing_time','duration','reaching_time','price','star_rating','seats_available'])
final_data.to_csv('G:/PROJECT - REDBUS/02_big_data.csv',index=False)